In [1]:
!pip install -q transformers datasets peft bitsandbytes accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 26.5 MB/s eta 0:00:00


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM,Trainer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer

In [3]:
from huggingface_hub import notebook_login
notebook_login()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
model_name = "google/gemma-2b"

from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
dataset = load_dataset("json", data_files="/content/sample_data/legal_dataset.jsonl")
dataset = dataset["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
def format_prompt(example):
    return {
        "text": f"""<start_of_turn>user
Simplify the following legal clause into plain English:

{example['input']}
<end_of_turn>
<start_of_turn>model
{example['target']}
<end_of_turn>"""
    }
dataset = dataset.map(format_prompt)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [9]:
#configure LORA
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj","v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.0368


In [16]:
training_args = TrainingArguments(
    output_dir="./gemma-legal",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=False,
    logging_steps=10,
    save_strategy="epoch",
    report_to="none",
    remove_unused_columns=False
)

In [20]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

Adding EOS to train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [21]:
trainer.train()

Step,Training Loss
10,3.589451
20,2.930987
30,2.264492
40,1.700576
50,1.393970
60,1.126686
70,0.981565
80,0.931109
90,0.860106
100,0.837220


TrainOutput(global_step=750, training_loss=0.7773968505859375, metrics={'train_runtime': 1477.9224, 'train_samples_per_second': 4.06, 'train_steps_per_second': 0.507, 'total_flos': 5205198903410688.0, 'train_loss': 0.7773968505859375})

In [22]:
# save model
trainer.model.save_pretrained("./gemma-legal-lora")
tokenizer.save_pretrained("./gemma-legal-lora")

('./gemma-legal-lora/tokenizer_config.json',
 './gemma-legal-lora/tokenizer.json')

In [27]:
# Test inference

prompt = """<start_of_turn>user
Simplify the following legal clause into plain English:

In witness whereof, the parties hereto have executed this Agreement as of the date first above written.
<end_of_turn>
<start_of_turn>model
"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.2
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

user
Simplify the following legal clause into plain English:

In witness whereof, the parties hereto have executed this Agreement as of the date first above written.

model
The parties that are mentioned in this clause have done everything that is in the next clause.

